# Setup for Amazon Bedrock
* Container: `conda_python3` <BR>
* We recommend `python 3.10` or later. 
    - version check: !python -V

## 0. Materials
- Bedrock user guide
    - https://preview.documentation.bedrock.aws.dev/Documentation/BedrockUserGuide.pdf

## 1. role setting (adding trust relationship)

### 1.1. role check

In [2]:
from sagemaker import get_execution_role

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: SageMakerExecutionRole-us-east-1-cdk


### 1.2. policy
- 1.1에서 확인된 롤에 아래와 같이 2개의 권한 추가

![nn](../imgs/policy.jpg)

### 1.3. policy
- 아래와 같이 인라인 정책 생성 (Add permissions - Create inline policy)
- policy name: bedrock
![nn](../imgs/inline-policy.jpg)

In [4]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "bedrock:*"
            ],
            "Resource": "*",
            "Effect": "Allow",
            "Sid": "BedrockFullAccess"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Action': ['bedrock:*'],
   'Resource': '*',
   'Effect': 'Allow',
   'Sid': 'BedrockFullAccess'}]}

### 1.4. Trust relationship
![nn](../imgs/trust-relationship.jpg)

In [5]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Principal': {'Service': 'sagemaker.amazonaws.com'},
   'Action': 'sts:AssumeRole'},
  {'Effect': 'Allow',
   'Principal': {'Service': 'bedrock.amazonaws.com'},
   'Action': 'sts:AssumeRole'}]}

## 2. Install python SDK for bedrock

In [6]:
!wget https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk.zip
!unzip bedrock-python-sdk.zip -d bedrock-sdk
!rm -rf bedrock-python-sdk.zip

--2023-08-24 09:59:04--  https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk.zip
Resolving d2eo22ngex1n9g.cloudfront.net (d2eo22ngex1n9g.cloudfront.net)... 108.138.82.216, 108.138.82.211, 108.138.82.207, ...
Connecting to d2eo22ngex1n9g.cloudfront.net (d2eo22ngex1n9g.cloudfront.net)|108.138.82.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 251046205 (239M) [application/zip]
Saving to: ‘bedrock-python-sdk.zip’

bedrock-python-sdk. 100%[===================>] 239.42M  97.4MB/s    in 2.5s    

2023-08-24 09:59:09 (97.4 MB/s) - ‘bedrock-python-sdk.zip’ saved [251046205/251046205]

Archive:  bedrock-python-sdk.zip
   creating: bedrock-sdk/reviews/
  inflating: bedrock-sdk/awscli-1.29.21.tar.gz  
  inflating: bedrock-sdk/AWSCLI32PY3.msi  
  inflating: bedrock-sdk/manifest.json  
  inflating: bedrock-sdk/AWSCLISetup.exe  
  inflating: bedrock-sdk/.functional  
  inflating: bedrock-sdk/awscli-bundle.zip  
  inflating: bedrock-sdk/boto3-1.28.

In [7]:
install_needed = True

In [8]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U ./bedrock-sdk/botocore-*-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/boto3-*-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/awscli-*-py3-none-any.whl
    !{sys.executable} -m pip install -U langchain
    !rm -rf bedrock-sdk

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Processing ./bedrock-sdk/botocore-1.31.21-py3-none-any.whl
botocore is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing ./bedrock-sdk/boto3-1.28.21-py3-none-any.whl
boto3 is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing ./bedrock-sdk/awscli-1.29.21-py3-none-any.whl
awscli is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


## 3. Check setting
아래 메시지가 뜨면 정상 <BR>
![nn](../imgs/check.png)


In [9]:
import boto3
import langchain

In [10]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")

langchain version check: 0.0.271
boto3 version check: 1.28.21


In [11]:
is_internal_use = True # 내부 직원 용
# is_internal_use = False # 고객 용

**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [12]:
#bedrock_region = "" ## <your region> 
bedrock_region = "us-east-1" 

In [13]:
if bedrock_region == "us-east-1":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://bedrock.us-east-1.amazonaws.com" if is_internal_use else None
    }
elif bedrock_region == "us-west-2":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://prod.us-west-2.frontend.bedrock.aws.dev" if is_internal_use else None
    }

In [14]:
bedrock_config

{'region_name': 'us-east-1',
 'endpoint_url': 'https://bedrock.us-east-1.amazonaws.com'}

In [15]:
if is_internal_use:
    bedrock_client = boto3.client(
        service_name='bedrock',
        region_name=bedrock_config["region_name"],
        endpoint_url=bedrock_config["endpoint_url"]
    )
else:
    bedrock_client = boto3.client(
        service_name='bedrock',
        region_name=bedrock_config["region_name"]
    ) 

In [16]:
output_text = bedrock_client.list_foundation_models()
output_text 

{'ResponseMetadata': {'RequestId': '82211390-8f3f-4cb3-8d66-b22f42aa25e7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 24 Aug 2023 09:59:35 GMT',
   'content-type': 'application/json',
   'content-length': '1166',
   'connection': 'keep-alive',
   'x-amzn-requestid': '82211390-8f3f-4cb3-8d66-b22f42aa25e7'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/stability.stable-diffusion-xl',
   'modelId': 'stability.stable-diffusion-xl'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-grande-instruct',
   'modelId': 'ai21.j2-grande-instruct'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-jumbo-instruct',
   'modelId': 'ai21.j2-jumbo-i

## 4. generation

In [17]:
from langchain.llms.bedrock import Bedrock

In [18]:
llm = Bedrock(
    model_id="amazon.titan-tg1-large",
    client=bedrock_client,
    model_kwargs={
        "maxTokenCount":4096,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    }
)

In [19]:
prompt = "Please let us know SageMaker's advantages in 100 words"

In [20]:
print (llm(prompt))


SageMaker is a fully managed service from Amazon that provides every developer and data scientist the best practices to build, train, and deploy high-quality machine learning (ML) models quickly. With SageMaker, there is no need to select and provision servers, configure ML infrastructure, or manage storage and security. Just use APIs to create, train, and deploy models fully managed in the cloud.
